# Comet ml

In [1]:
# $ export COMET_API_KEY="ZNgNJ1VVgmaAbL0ga1t4mw3JI"

In [2]:
# !comet check

In [1]:
import comet_ml

# Arguments

In [2]:
model_checkpoint = "microsoft/deberta-v3-large"
SEED = 42

# Tokenizer

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/tanluuuuuuu/miniconda3/envs/one_for_all/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Data loader

In [13]:
ner_tags = [
"MAT",
"NMAT",
"DIMENSION",
"WEIGHT",
"TARGET_USER",
"PROPERTY",
"COLOR",
"SHAPE",
"SIZE",
]

processed_ner_tags = ['O']
for tag in ner_tags:
        processed_ner_tags.extend([f"B-{tag}", f"I-{tag}"])

print(processed_ner_tags)
print(len(processed_ner_tags))

ner_tags_2_number = {t: i for (i, t) in enumerate(processed_ner_tags)}
number_2_ner_tags = {t: i for (t, i) in enumerate(ner_tags_2_number)}

['O', 'B-MAT', 'I-MAT', 'B-NMAT', 'I-NMAT', 'B-DIMENSION', 'I-DIMENSION', 'B-WEIGHT', 'I-WEIGHT', 'B-TARGET_USER', 'I-TARGET_USER', 'B-PROPERTY', 'I-PROPERTY', 'B-COLOR', 'I-COLOR', 'B-SHAPE', 'I-SHAPE', 'B-SIZE', 'I-SIZE']
19


In [14]:
import pandas as pd

raw_excel_dataset = pd.read_excel("../data/data_from_09_10_2023/raw_data_restore_uppercase_11_1_2023.xlsx")
raw_excel_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4640 entries, 0 to 4639
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  4640 non-null   object
 1   locs      4640 non-null   object
 2   words     4640 non-null   object
dtypes: object(3)
memory usage: 108.9+ KB


In [15]:
import ast

def assign_ner_tags_deberta(example):
    # print(example['sentence'])
    token_input = tokenizer(example['sentence'])
    example['tokens'] = tokenizer.convert_ids_to_tokens(
        token_input['input_ids'])

    # print(example['sentence'])
    # print(example['tokens'])
    # print("len: ", len(example['tokens']))

    ner_tags = [0 for token in example['tokens']]
    if str(type(example['locs'])) == "<class 'list'>":
        locs = example['locs']
    else:
        locs = ast.literal_eval(example['locs'])

    locs = [(int(loc[0]), int(loc[1]), loc[2]) for loc in locs]
    locs = sorted(locs)
    bg_id = 1
    pre_loc = 0
    text = example['sentence']
    for loc in locs:
        loc0 = int(loc[0])
        loc1 = int(loc[1])

        pre_text = text[pre_loc:loc0]
        token_input = tokenizer(pre_text)
        pre_token = tokenizer.convert_ids_to_tokens(
            token_input['input_ids'])

        # print(pre_text)
        # print(pre_token)
        # print("len: ", len(pre_token))

        bg_id = bg_id + len(pre_token) - 2
        pre_loc = loc1

        # print("bg_id: ", bg_id)

        word = example['sentence'][loc0: loc1]
        token_input = tokenizer(word)
        word_token = tokenizer.convert_ids_to_tokens(token_input['input_ids'])

        label_number = ner_tags_2_number[f"B-{loc[2]}"]
        ner_tags[bg_id] = label_number
        bg_id += 1
        for idx in range(bg_id, bg_id + len(word_token) - 3):
            ner_tags[idx] = label_number + 1
        bg_id = bg_id + len(word_token) - 3

        # visualize_ner_tags(example['tokens'], ner_tags)

    ner_tags[0] = -100
    ner_tags[-1] = -100
    return ner_tags


In [16]:
from datasets import Dataset, DatasetDict, ClassLabel, Features, Value, Sequence

list_input_ids = []
list_attention_mask = []
list_labels = []

for index, row in raw_excel_dataset.iterrows():
    try:
        label = assign_ner_tags_deberta(row)
        token_input = tokenizer(row['sentence'])
        list_input_ids.append(token_input['input_ids'])
        list_attention_mask.append(token_input['attention_mask'])
        list_labels.append(label)
    except Exception as error:
        print(error)
        break
        print(index)
        print(row['sentence'])

tokenized_datasets = pd.DataFrame()
tokenized_datasets['input_ids'] = pd.Series(list_input_ids)
tokenized_datasets['attention_mask'] = pd.Series(list_attention_mask)
tokenized_datasets['labels'] = pd.Series(list_labels)

train_dataset = Dataset.from_pandas(tokenized_datasets)
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4640
})

In [17]:
tokenized_datasets = train_dataset.train_test_split(test_size=0.2, shuffle=True, seed=7)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3712
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 928
    })
})

In [18]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=4,
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=4
)

# Train model

In [19]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [21]:
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
import numpy as np

def get_class_weight(train_dataset, num_class):
    list_label = []
    for label in train_dataset['labels']:
        for cl in label:
            if cl != -100:
                list_label.append(cl)
    class_weight = compute_class_weight(class_weight='balanced', classes=np.arange(num_class), y=list_label)
    return class_weight

class_weight = get_class_weight(tokenized_datasets["train"], len(processed_ner_tags))
print(class_weight)
print(len(class_weight))

[6.39918747e-02 3.88201359e+00 8.30435028e+00 1.11280677e+01
 1.04814367e+02 5.13319865e+00 8.70609850e-01 2.05736423e+01
 1.26943751e+01 6.97505680e+00 2.54303710e+01 1.97762957e+00
 1.67485708e+00 1.85113679e+01 1.58291085e+02 3.16582170e+01
 4.30903509e+01 4.87815293e+01 2.67457350e+02]
19


## Model classes

In [22]:
# https://www.kaggle.com/code/wuharlem/simple-bert-w-hinge-loss

from transformers import RobertaForTokenClassification, DebertaV2ForTokenClassification
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import TokenClassifierOutput
from torch.nn import CrossEntropyLoss
import torch
 
class CustomDebertaV2(DebertaV2ForTokenClassification):
    def __init__(self, config, class_weight):
        super().__init__(config)
        self.class_weight = torch.tensor(class_weight, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
        
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.deberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            if self.class_weight is None:
                loss_fct = CrossEntropyLoss()
            else:
                loss_fct = CrossEntropyLoss(weight=self.class_weight)
                
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions
        )

## Setup training

In [23]:
from transformers import AutoModelForTokenClassification
from transformers.models.bert import modeling_bert
from transformers import RobertaForTokenClassification, DebertaV2ForTokenClassification

label_names = processed_ner_tags
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model = CustomDebertaV2.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    class_weight=class_weight
)

print(model.config)

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing CustomDebertaV2: ['lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing CustomDebertaV2 from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomDebertaV2 from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFo

DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MAT",
    "2": "I-MAT",
    "3": "B-NMAT",
    "4": "I-NMAT",
    "5": "B-DIMENSION",
    "6": "I-DIMENSION",
    "7": "B-WEIGHT",
    "8": "I-WEIGHT",
    "9": "B-TARGET_USER",
    "10": "I-TARGET_USER",
    "11": "B-PROPERTY",
    "12": "I-PROPERTY",
    "13": "B-COLOR",
    "14": "I-COLOR",
    "15": "B-SHAPE",
    "16": "I-SHAPE",
    "17": "B-SIZE",
    "18": "I-SIZE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-COLOR": 13,
    "B-DIMENSION": 5,
    "B-MAT": 1,
    "B-NMAT": 3,
    "B-PROPERTY": 11,
    "B-SHAPE": 15,
    "B-SIZE": 17,
    "B-TARGET_USER": 9,
    "B-WEIGHT": 7,
    "I-COLOR": 14,
    "I-DIMENSION": 6,
    "I-MAT": 2,
    "I-NMAT": 4,
    "I-PROPERTY": 12,
    "I-SHAPE": 16,
    "I-SIZE": 18,
    "I-TARG

In [24]:
import evaluate
import numpy as np
from torch.optim import AdamW
from transformers import get_scheduler

metric = evaluate.load("seqeval")
optimizer = AdamW(model.parameters(), lr=2e-5)

num_train_epochs = 100
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [26]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, test_dataloader
)

In [27]:
from comet_ml import Experiment, ExistingExperiment
from comet_ml.integration.pytorch import log_model

experiment = Experiment(
    api_key="ZNgNJ1VVgmaAbL0ga1t4mw3JI",
    project_name="deberta-v3-large-ner-attribution-extraction",
    workspace="luunvt",
    log_code=True
)

hyper_params = {
    "model": model_checkpoint, 
    "num_epochs": num_train_epochs, 
    "optimizer": "adamW",
    "use_class_weight": True,
    "num_train_sample": len(tokenized_datasets["train"]),
    "num_test_sample": len(tokenized_datasets["test"]),
}
experiment.log_parameters(hyper_params)
experiment.add_tags(["deberta-ner-classweight"])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


COMET INFO: Experiment is live on comet.com https://www.comet.com/luunvt/deberta-v3-large-ner-attribution-extraction/57573240eb1143bf89e203b8b076ab4c



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
import comet_ml
from tqdm.auto import tqdm
import torch
from datetime import datetime
import os
import logging

date_time = datetime.now()
format_date = date_time.strftime('%Y-%m-%d')
format_time = date_time.strftime('%H:%M:%S')

print(f"Date: {format_date}")
print(f"Time: {format_time}")

output_dir = f"../models/model_from_{format_date}/deberta-v3-large-_{format_time}"

best_f1_score = 0
    
for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        
    with experiment.test():
        # Evaluation
        model.eval()
        for batch in test_dataloader:
            with torch.no_grad():
                outputs = model(**batch)

            predictions = outputs.logits.argmax(dim=-1)
            labels = batch["labels"]

            # Necessary to pad predictions and labels for being gathered
            predictions = accelerator.pad_across_processes(
                predictions, dim=1, pad_index=-100)
            labels = accelerator.pad_across_processes(
                labels, dim=1, pad_index=-100)

            predictions_gathered = accelerator.gather(predictions)
            labels_gathered = accelerator.gather(labels)

            true_predictions, true_labels = postprocess(
                predictions_gathered, labels_gathered)
            metric.add_batch(predictions=true_predictions, references=true_labels)

        results = metric.compute()
        print(
            f"epoch {epoch}:",
            {
                key: results[f"overall_{key}"]
                for key in ["precision", "recall", "f1", "accuracy"]
            },
        )

        if experiment:
            experiment.set_epoch(epoch)
            experiment.log_metric("precision", results["overall_precision"])
            experiment.log_metric("recall", results["overall_recall"])
            experiment.log_metric("f1", results["overall_f1"])
            experiment.log_metric("accuracy", results["overall_accuracy"])

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process and results[f"overall_f1"] > best_f1_score:
        output_ckpt = os.path.join(output_dir, f'best_f1')
        best_f1_score = results[f"overall_f1"]
        print(
            f"Save best f1 model at epoch {epoch} with better f1 score {best_f1_score}")
        tokenizer.save_pretrained(output_dir)
        unwrapped_model.save_pretrained(
            output_ckpt, save_function=accelerator.save)
    if (epoch + 1) % 5 == 0:
        output_ckpt = os.path.join(output_dir, f'epoch_{epoch + 1}')
        print(f"Save model at epoch {epoch + 1}")
        tokenizer.save_pretrained(output_ckpt)
        unwrapped_model.save_pretrained(
            output_ckpt, save_function=accelerator.save)
experiment.end()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Date: 2023-11-02
Time: 09:31:05
epoch 0: {'precision': 0.6870229007633588, 'recall': 0.287636669470143, 'f1': 0.4055015413801281, 'accuracy': 0.835397634926862}
Save best f1 model at epoch 0 with better f1 score 0.4055015413801281
epoch 1: {'precision': 0.6842105263157895, 'recall': 0.32984698818516367, 'f1': 0.4451123889179299, 'accuracy': 0.864652805021577}
Save best f1 model at epoch 1 with better f1 score 0.4451123889179299
epoch 2: {'precision': 0.7175572519083969, 'recall': 0.37623762376237624, 'f1': 0.49364289662797123, 'accuracy': 0.8783836798744605}
Save best f1 model at epoch 2 with better f1 score 0.49364289662797123
epoch 3: {'precision': 0.7605464041783849, 'recall': 0.44932352243057205, 'f1': 0.5649059982094897, 'accuracy': 0.9004371462198061}
Save best f1 model at epoch 3 with better f1 score 0.5649059982094897
epoch 4: {'precision': 0.6998794696665327, 'recall': 0.39067055393586003, 'f1': 0.501439263097294, 'accuracy': 0.8881073810457882}
Save model at epoch 5
epoch 5: 

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/luunvt/deberta-v3-large-ner-attribution-extraction/57573240eb1143bf89e203b8b076ab4c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [9280]          : (4.944258307659766e-06, 4.1543660163879395)
COMET INFO:     test_accuracy [100]  : (0.7681443703413103, 0.9599843075715967)
COMET INFO:     test_f1 [100]        : (0.27993955421231587, 0.7739949990382767)
COMET INFO:     test_precision [100] : (0.5954198473282443, 0.8308557653676175)
COMET INFO:     test_recall [100]    : (0.18298555377207062, 0.7493403693931399)
COMET INFO:   Parameters:
COMET INFO:     model            : microsoft/deberta-v3-large
COMET INFO:     n